# importation des bibliothéque

On a importer *`MLPClassifier`* from *`sklearn.neural_network`* pour créer un multi layer perceptron pour entraîner des réseaux de neurones artificiels pour la classification,on a aussi importer *`train_test_split`* from *`sklearn.model_selection`* pour deviser la dataset en train et test datasets, aprés on a importer *`dataset`* from *sklearn* car le module *`sklearn.dataset`* contient des dataset.
On a importer *`base`* from *`deap`* pour fournir des éléments de base nécessaires pour utiliser la bibliothèque DEAP, *`creator`* pour créer des nouveaux types d'objets évolutifs,*`tools`* pour fournir des outils pour effectuer des operations courantes dans les algorithme évolutifs et *`algorithms`* qui contient des algo évolutifs prédéfinis



In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np
from sklearn.metrics import accuracy_score
from deap import base, creator, tools, algorithms

# importation de dataset et devision en test et train 
on a charger la dataset iris en utilisants la fonction *`load_iris()`* de la bibliothéque *datasets*.
aprés on a divisé la dataset en un ensemle de *train* et ensemble de *test* en utilisant *`train_test_split()`*.
iris.data contient les caractéristiques (longueur et largeur des sépales et des pétales) pour chaque échantillon.
iris.target contient les étiquettes de classe (0 pour setosa, 1 pour versicolor et 2 pour virginica) pour chaque échantillon.

In [22]:
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)


# Fonction d'évaluation pour l'optimisation d'un modèle de réseau de neurones multicouches (MLP)




In [23]:
def evaluate(individual):
    learning_rate = individual[0]
    # Création du classifieur MLP avec le taux d'apprentissage spécifié
    clf = MLPClassifier(hidden_layer_sizes=(13, 13, 3), max_iter=15000, learning_rate='adaptive',
                        learning_rate_init=learning_rate, random_state=42)
    clf.fit(X_train, y_train)
    # Prédiction sur l'ensemble de test
    y_pred = clf.predict(X_test)
    # Calcul de l'accuracy
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy,


# Définition de types d'objets évolutifs dans DEAP
on a crée un nouveau type d'objet qui s'appel *FItnessMax* en se basent sur *base.Fitness* pour représenter le fitness d'un individu, dans ce cas elle est pondérée avec un poids de 1.0 ce que signifie qu'elle est utiliser pour maximiser une fonction lors de l'optimisation.
on a aussi crée un nouveau type d'objet appelé *Individual*. Cet objet est une liste qui représente un individu dans l'algorithme évolutif. La fitness de chaque individu est associée au type "FitnessMax" que nous avons déja créé. Cela signifie que chaque individu de type "Individual" aura une valeur de fitness qui peut être utilisée pour évaluer sa qualité dans le contexte de l'algorithme évolutif.

In [24]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


C:\Users\LENOVO\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\LENOVO\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


# création de `toolbox` et l'enregistrement des opérateurs
On a crée une boîte à outils *toolbox* vide à l'aide de la classe **`base.Toolbox()`**,cette boite pour enregistrer les opérateurs dénétiques et d'autres fonctions nécessaires.
Aprés la création de la boite et à l'aide de **`toolbox.register("attr_float", np.random.uniform, 0.0001, 1.0)`**, on a enregistrer une fnction appelée *attr_float* , cette fonction est pour énérer des valeurs al"atoire de la plage [0.0001, 1.0].
on a aussi enregistrer une fonction qui s'appelle *individual* en utilisant **`toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=1)`** ,Cette fonction crée un individu de type "Individual" en appelant la fonction *attr_float* une fois. Cela signifie que chaque individu de la population aura un seul attribut, qui sera un nombre généré aléatoirement de *attr_float*.
on a enregistrer une fonction appelée *population* dans la boite *toolbox* à l'aide de **`toolbox.register("population", tools.initRepeat, list, toolbox.individual)`** Cette fonction crée une population en répétant l'appel à la fonction "individual" pour créer plusieurs individus et les stocker dans une liste.
**`toolbox.register("evaluate", evaluate)`** , Cela enregistre la fonction d'évaluation . Cette fonction sera utilisée pour évaluer la fitness de chaque individu de la population.
et pour enregistrer l'opérateur de croisement on a utiliser **`toolbox.register("mate", tools.cxBlend, alpha=0.5)`**,cet opérateur utilisé pour croiser les individus lors de la génération.
on a enregistrer l'opérateur de mutation dans la boite *toolbox* en utilisant **`toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)`** pour muter les individus de la population en introduisant des variations aléatoires dans leurs attributs.
et finalement on a enragistrer l'operateur de selection **`toolbox.register("select", tools.selTournament, tournsize=3)`**, cet operateur est pour séléctionner les individus à reproduire lors de la génération en utilisant un tournoi de taille 3 pour comparer les individus.


In [25]:
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 0.0001, 1.0)  # Learning rate range: [0.0001, 1.0]
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Définition du fonction main

on a crée une population de 50 individus,et un objetHallOfFame pour stocker le meilleur individus rencontrée, et quelque statistics functions pour mesurer la performance des individus, puis on a initailiser quelque variables on va l'utiliser par la suite.
dans une boucle qui continue jusqu'à la génération=50, à chaque itération de la boucle, de nouveaux individus sont générés en utilisant l'opérateur de variation *`varOr`*. Cela peut inclure la reproduction, le croisement et la mutation des individus de la population actuelle, la fonction d'évaluation est utilisée pour évaluer la fitness de chaque individu de la population et les individus sont ensuite mis à jour avec leurs valeurs de fitness , si la fitness de l'un des individus dépasse la meilleure précision enregistrée jusqu'à présent, la meilleure précision et le taux d'apprentissage correspondant sont mis à jour.

In [26]:
def main():
    pop = toolbox.population(n=4)
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=True)
    best_individual = tools.selBest(pop, k=1)[0]
    best_learning_rate = best_individual[0]
    print("Meilleur taux d'apprentissage trouvé:", best_learning_rate)

    

In [27]:
if __name__ == "__main__":
    main()

gen	nevals
0  	4     
1  	4     
2  	3     
3  	2     
4  	2     
5  	1     
6  	2     
7  	4     
8  	3     
9  	4     
10 	2     
11 	3     
12 	2     
13 	1     
14 	4     
15 	2     
16 	0     


ValueError: learning_rate_init must be > 0, got adaptive.